# Input

The input is time-series forecasted that give the number of transfers started for eight days. We have an anomaly represented by the number of transfers plunged on February 16th at 5:30 am and bottoming out during 5 hours.

In the first section, we load the forecasted and display the curve of transfers started.

In [6]:
import pandas as pd
from pandas.core.frame import DataFrame
import plotly.graph_objects as go
import plotly.express as px

#forecasted for the model
df = pd.read_csv('../forecasted/start-tranfers.csv')

#render
fig = go.Figure()
#display model
fig.add_trace(go.Scatter(
    x=df.ds,
    y=df.y,
    line_color='rgba(51, 81, 184, 0.8)',
    name="Start tranfer"))
fig.show()

# Configure Prophet algorithm

We use the [Prophet](https://facebook.github.io/prophet/) algorithm to analyse the time-series forecasted. Then, we train the model and predict the forecasted with the same input.

In [7]:
from prophet import Prophet
from prophet.plot import add_changepoints_to_plot

#Configure prophet engine
prophet = Prophet()
#train
model = prophet.fit(df)
#run
forecasted = model.predict(df)
forecasted['fact'] = df['y'].reset_index(drop = True)

#render
fig = go.Figure()
#display model
fig.add_trace(go.Scatter(
    x=forecasted.ds,
    y=forecasted.yhat,
    line_color='rgba(51, 81, 184, 0.8)',
    name="yhat"))
fig.add_trace(go.Scatter(
    x=forecasted.ds,
    y=forecasted.yhat_upper,
    line_color='rgba(51, 81, 184, 0.5)',
    customdata=forecasted.yhat_upper - forecasted.yhat,
    hovertemplate='yhat_upper:%{y:.3f}<br>ref:%{customdata:.2f}',
    name="yhat_upper"))
fig.add_trace(go.Scatter(
    x=forecasted.ds,
    y=forecasted.yhat_lower,
    customdata=forecasted.yhat - forecasted.yhat_lower,
    hovertemplate='yhat_upper:%{y:.3f}<br>ref:%{customdata:.2f}',
    fill='tonexty',
    line_color='rgba(51, 81, 184, 0.5)',
    name="yhat_lower"))
#display fact
fig.add_trace(go.Scatter(
    x=forecasted.ds,
    y=forecasted.fact,
    line_color='rgba(148, 242, 119, 1)',
    name="fact"))
fig.show()

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.


Initial log joint probability = -20.831
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      67       1604.12   9.79998e-05       67.6619   1.397e-06       0.001      129  LS failed, Hessian reset 
      99       1604.14   8.56352e-05       74.3953           1           1      170   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     161       1604.15   6.35551e-09        73.759     0.07329     0.07329      256   
Optimization terminated normally: 
  Convergence detected: absolute parameter change was below tolerance


# Detect the outliers

We want to detect the outliers, especially when we have lower than transfers as expected.  

> We don't care if the number of transfers is more significant than expected

To detect the anomaly we calculate the distance of the outlier compare of the interval range of the model.

In [23]:
#detect the outliers
forecasted['anomaly'] = 0
forecasted.loc[forecasted['fact'] > forecasted['yhat_upper'], 'anomaly'] = 1
forecasted.loc[forecasted['fact'] < forecasted['yhat_lower'], 'anomaly'] = -1

#calculate importance
forecasted['importance'] = 0
forecasted.loc[forecasted['anomaly'] ==1, 'importance'] = \
    (forecasted['fact'] - forecasted['yhat_upper'])/(forecasted['yhat_upper'] - forecasted['yhat'])
forecasted.loc[forecasted['anomaly'] ==-1, 'importance'] = \
    (forecasted['yhat_lower'] - forecasted['fact'])/(forecasted['yhat'] - forecasted['yhat_lower'])

#categorize anomalies
forecasted.loc[forecasted['importance'] < .25, 'category'] = 'warning'
forecasted.loc[(forecasted['importance'] >= .25) & (forecasted['importance'] < .50), 'category'] = 'minor'
forecasted.loc[(forecasted['importance'] >= .50) & (forecasted['importance'] < .75), 'category'] = 'major'
forecasted.loc[forecasted['importance'] >= .75, 'category'] = 'critical'

#filter the outliers that lower than the model
anomalies = forecasted[forecasted['anomaly'] ==-1]

#render the anomalies
fig = go.Figure()
#display model
fig.add_trace(go.Scatter(
    x=forecasted.ds,
    y=forecasted.yhat,
    line_color='rgba(51, 81, 184, 0.8)',
    name="yhat"))
fig.add_trace(go.Scatter(
    x=forecasted.ds,
    y=forecasted.yhat_upper,
    line_color='rgba(51, 81, 184, 0.5)',
    customdata=forecasted.yhat_upper - forecasted.yhat,
    hovertemplate='yhat_upper:%{y:.3f}<br>ref:%{customdata:.2f}',
    name="yhat_upper"))
fig.add_trace(go.Scatter(
    x=forecasted.ds,
    y=forecasted.yhat_lower,
    customdata=forecasted.yhat - forecasted.yhat_lower,
    hovertemplate='yhat_upper:%{y:.3f}<br>ref:%{customdata:.2f}',
    fill='tonexty',
    line_color='rgba(51, 81, 184, 0.5)',
    name="yhat_lower"))
#display fact
fig.add_trace(go.Scatter(
    x=forecasted.ds,
    y=forecasted.fact,
    line_color='rgba(148, 242, 119, 1)',
    name="fact"))
#display anomalies
if(anomalies.size > 0):
    fig_anomalies = px.scatter(anomalies,
        x="ds",
        y="fact",
        color="category",
        hover_data={
            'importance':':.2f'},
        color_discrete_map={
                "warning": "#008cd1",
                "minor": "yellow",
                "major": "orange",
                "critical": "red"})
    fig.add_traces(fig_anomalies.data)
fig.update_layout(hovermode='x unified')
fig.show()

ValueError: 
    Invalid element(s) received for the 'data' property of 
        Invalid elements include: [Figure({
    'data': [{'customdata': array([[0.45594486],
                                   [0.31287967],
                                   [0.43066364],
                                   [0.31853243],
                                   [0.34130606],
                                   [0.27483245],
                                   [0.25154518],
                                   [0.33473784],
                                   [0.35050992]]),
              'hovertemplate': ('category=minor<br>ds=%{x}<br>f' ... 'tomdata[0]:.2f}<extra></extra>'),
              'legendgroup': 'minor',
              'marker': {'color': 'yellow', 'symbol': 'circle'},
              'mode': 'markers',
              'name': 'minor',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'x': array([datetime.datetime(2021, 2, 9, 19, 15),
   